In [1]:
import sys
import torch as th
import argparse
import os
import numpy as np
import torch as th
import torch.distributed as dist
import cv2
import csv
import tqdm
sys.path.insert(0, "../../")
from utils.metrics import PSNR, SSIM, LPIPS
from utils import logger

/home/yokoyama/.pyenv/versions/anaconda3-2023.07-2/envs/IRS/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def write_scores_to_csv(out_path, start_step, pre_model, scores):
    header = ['pre-model', 'start step'] + list(scores.keys())
    mode = 'w' if not os.path.isfile(out_path) else 'a'
    with open(out_path, mode, newline='') as f:
        writer = csv.writer(f)
        if mode == 'w':
            writer.writerow(header)
        writer.writerow([pre_model] + [start_step] + [i[0] for i in scores.values()])

def reverseChannel(img):
  # 画像コピー
  dst = img.copy()
  # チャネル入れ替え
  dst[:, :, 0] = img[:, :, 2]
  dst[:, :, 2] = img[:, :, 0]

  return dst

def compute_score(all_images_gt_raw, all_images_pred_raw, start_step, pre_model):
    metrics = ('psnr', 'ssim', 'lpips')
    device = 'cpu'
    boundary_ignore = 40
    metrics_all = {}
    scores = {}
    for m in ["psnr","ssim","lpips"]:
        if m == 'psnr':
            loss_fn = PSNR(boundary_ignore=boundary_ignore)
        elif m == 'ssim':
            loss_fn = SSIM(boundary_ignore=boundary_ignore, use_for_loss=False)
        elif m == 'lpips':
            loss_fn = LPIPS(boundary_ignore=boundary_ignore)
            loss_fn.to(device)
        else:
            raise ValueError(f"Unknown metric: {m}")
        metrics_all[m] = loss_fn
        scores[m] = []

    scores = {k: [] for k, v in scores.items()}
    all_images_gt_raw = th.cat(all_images_gt_raw)
    all_images_pred_raw = th.cat(all_images_pred_raw)

    for m, m_fn in metrics_all.items():
        metric_value = m_fn(all_images_pred_raw, all_images_gt_raw).cpu().item()
        scores[m].append(metric_value)
        logger.log(f"{m} is {metric_value}")

    out_path = os.path.join("official/lpips", f"score.csv")
    write_scores_to_csv(out_path, start_step, pre_model, scores)

In [3]:
gt_path = "/home/yokoyama/image-restoration-sde/codes/config/deshadow/official/gt"
pre_path = "/home/yokoyama/image-restoration-sde/codes/config/deshadow/official/timestep5"
all_images_gt_raw = []
all_images_pred_raw = []
#img_list = os.listdir(gt_path)
img_list = os.listdir(gt_path)
for img in img_list: 
    gt = os.path.join(gt_path,img)
    pre = os.path.join(pre_path,img)
    gt_img = cv2.imread(gt, cv2.IMREAD_UNCHANGED).astype(np.float32) / 255.
    pre_img = cv2.imread(pre, cv2.IMREAD_UNCHANGED).astype(np.float32) / 255.
    gt_img = th.unsqueeze(th.from_numpy(np.ascontiguousarray(np.transpose(gt_img, (2, 0, 1)))).float(), dim=0)[:, [2, 1, 0]]
    pre_img = th.unsqueeze(th.from_numpy(np.ascontiguousarray(np.transpose(pre_img, (2, 0, 1)))).float(), dim=0)[:, [2, 1, 0]]
    all_images_gt_raw.append(gt_img)
    all_images_pred_raw.append(pre_img)
compute_score(all_images_gt_raw, all_images_pred_raw, start_step="5_T", pre_model="unknown")

Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]


/home/yokoyama/.pyenv/versions/anaconda3-2023.07-2/envs/IRS/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/yokoyama/.pyenv/versions/anaconda3-2023.07-2/envs/IRS/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Loading model from: /home/yokoyama/.pyenv/versions/anaconda3-2023.07-2/envs/IRS/lib/python3.10/site-packages/lpips/weights/v0.1/alex.pth
Logging to /tmp/bsrd-2024-03-21-10-17-23-350664/bsrd-2024-03-21-10-17-23-350754
psnr is 28.048084259033203
ssim is 0.8622182011604309
lpips is 0.037064939737319946
